In [16]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score

In [17]:
# 1. 교통량 데이터 로드
# 예시 파일 경로를 사용하여 엑셀 데이터를 불러옵니다.
weekdays_data = pd.read_excel('../data/weekday_traffic.xlsx')

In [18]:
# 2. 독립 변수: 각 날의 8시, 9시, 10시 교통량 데이터를 사용
# '8시', '9시', '10시' 열을 독립 변수로 선택하여 교통량 변수를 구성합니다.
X = weekdays_data[['8시', '9시', '10시']]

In [19]:
# 3. 종속 변수: 혼잡 여부 (True/False -> 1/0 변환)
# '혼잡' 열을 종속 변수로 사용하며, 이 값이 True이면 1, False이면 0으로 변환합니다.
y = weekdays_data['혼잡'].astype(int)

In [20]:
# 4. 데이터 분리 (훈련 데이터 80%, 테스트 데이터 20%)
# 데이터를 학습용과 테스트용으로 나눕니다.
# 참고: Scikit-learn - train_test_split: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [21]:
# 5. 데이터 스케일링 (로지스틱 회귀에 반드시 필요)
# 데이터의 스케일에 민감한 로지스틱 회귀 모델에 적용하기 때문에 표준화를 진행합니다. (상대적으로 트리 기반 모델에서는 덜 민감하다는거지 하면 안되는 것은 아닙니다.)
# 앙상블 모델은 스케일링 없이 원본 데이터를 사용합니다.
# 참고: Scikit-learn - StandardScaler: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [22]:
# (1) 랜덤 포레스트 - 배깅 방식
# 배깅(bagging) 방식의 랜덤 포레스트 모델을 생성합니다. 
# - n_estimators=10: 트리의 개수를 10개로 설정
# - random_state=42: 결과 재현을 위해 고정된 난수 시드를 설정
# 참고: Scikit-learn - RandomForestClassifier: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
rf_model = RandomForestClassifier(n_estimators=10, random_state=42)

In [23]:
# (2) Gradient Boosting - 부스팅 방식
# 부스팅 방식의 Gradient Boosting 모델을 생성합니다.
# - n_estimators=10: 트리 개수를 10개로 설정
# - learning_rate=0.1: 학습률을 0.1로 설정하여 학습 속도 조절
# - max_depth=3: 각 트리의 최대 깊이를 3으로 설정하여 과적합 방지
# 참고: Scikit-learn - GradientBoostingClassifier: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
gb_model = GradientBoostingClassifier(n_estimators=10, learning_rate=0.1, max_depth=3, random_state=42)

In [24]:
# (3) Logistic Regression - 로지스틱 회귀 모델
# Logistic Regression 모델을 생성하며 L2 정규화를 사용합니다.
# - penalty='l2': L2 정규화(릿지 회귀)를 사용
# - C=1.0: 정규화 강도 조절, C 값이 클수록 약한 정규화, 작을수록 강한 정규화
# 참고: Scikit-learn - LogisticRegression: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
lr_model = LogisticRegression(C=1.0, penalty='l2', random_state=42)


In [25]:
# 7. 모델 학습 및 성능 평가
# 설정한 세 가지 모델에 대해 학습을 수행하고, 테스트 데이터에 대한 성능을 평가합니다.
models = {
    'Random Forest': rf_model,
    'Gradient Boosting': gb_model,
    'Logistic Regression': lr_model
}


In [26]:
# 각 모델에 대해 학습 및 평가
for model_name, model in models.items():
    if model_name == 'Logistic Regression':
        # Logistic Regression은 스케일링된 데이터를 사용합니다.
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        # 앙상블 모델들은 원본 데이터를 사용해 학습합니다.
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
    # 모델의 정확도와 분류 리포트를 출력합니다.
    # 참고: Scikit-learn - classification_report: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    
    print(f"\n==== {model_name} ====")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Classification Report:\n{report}")


==== Random Forest ====
Accuracy: 0.89
Classification Report:
              precision    recall  f1-score   support

           0       0.86      1.00      0.92         6
           1       1.00      0.67      0.80         3

    accuracy                           0.89         9
   macro avg       0.93      0.83      0.86         9
weighted avg       0.90      0.89      0.88         9


==== Gradient Boosting ====
Accuracy: 0.78
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.83      0.83         6
           1       0.67      0.67      0.67         3

    accuracy                           0.78         9
   macro avg       0.75      0.75      0.75         9
weighted avg       0.78      0.78      0.78         9


==== Logistic Regression ====
Accuracy: 0.78
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.83      0.83         6
           1       0.67      0.67     